# Data Augmentation Experiments

### Prepare the images with the associated experiments for 1, 3, 5, 10, 20 and all shot learning 

In [ ]:
import torch
import torch.nn as nn
import numpy as np
import torchvision
import string
import os
import copy
import shutil
import pandas as pd
from PIL import Image
from random import sample
from torchvision import datasets, models, transforms

import pandas as pd
import os.path, sys 
from pathlib import Path

In [ ]:
torch.manual_seed(0)
np.random.seed(0)

In [ ]:
# Save Image ID, whether to train or test image, Folder Name, Name of Image in Data Frame
main_dir = '/Users/madisonvanhorn/Documents/iNaturalist_2017/CUB_200_2011/CUB_200_2011/'
part_dir = main_dir + 'parts/' + 'part_locs.txt'
mirror_dir = main_dir + 'mirror/' 
df = pd.read_csv(part_dir, delimiter = ' ', names=['ID', 'Part_ID', 'X', 'Y', 'Visible'])

diff_parts = main_dir + 'parts/' + 'parts.txt'
parts_df = pd.read_csv(diff_parts, delimiter = ' ', names=['ID', 'Part'])

In [ ]:
img_txt = main_dir + 'images.txt'
img_split = main_dir + 'train_test_split.txt'
img_df =  pd.read_csv(img_txt, delimiter = ' ', names=['ID', 'Name'])
img_split_df = pd.read_csv(img_split, delimiter = ' ', names=['ID', 'Training'])
img_df = img_df.join(img_split_df['Training'])

## Transfer Images to perspective Train/Test folder

In [ ]:
# Move images to their perspective folders
for index, row in df.iterrows():
    if(os.path.isdir(train_dir)==False):
            os.mkdir(train_dir)
    if(os.path.isdir(test_dir)==False):
        os.mkdir(test_dir)

    if (row[1]==1): # move to training folder
        if (os.path.isdir(train_dir + row[2])==False):
            os.mkdir(train_dir + row[2])
        shutil.copy(img_dir + row[2] + '/' + row[3], train_dir + row[2] + '/' + row[3]) 

    else: # Move to test folder
        if (os.path.isdir(test_dir + row[2])==False):
            os.mkdir(test_dir + row[2])
        shutil.copy(img_dir + row[2] + '/' + row[3], test_dir  + row[2] + '/' + row[3]) 

## Bounding Box

In [ ]:
# boundaries.csv contains the meta-data of each image
# path/to/folder/image_name.jpg
# x, y co-ordinates and height and width of the bounding boces
# if the image is present in train set or test set, etc.

boundary_df = pd.read_csv("boundaries.csv", )

In [ ]:
path = os.getcwd()

In [ ]:
for i in range(len(boundary_df)):

    im = Image.open(os.path.join(path, boundary_df['folder'][i], boundary_df['name'][i]))
    # crop the image as per given co-ordinates and measurements
    imCrop = im.crop((boundary_df['x'][i], boundary_df['y'][i],
                      boundary_df['x'][i] + boundary_df['width'][i],
                      boundary_df['y'][i] + boundary_df['height'][i])) #corrected
    # store in test set if it is a test image else store in train
    if df['train'][i] == 0: 

        if os.path.exists(os.path.join(path, 'cropped', 'test', df['folder'][i])):
            im_crop.save(os.path.join(path, 'cropped', 'test', df['folder'][i],
                                              df['name'][i]), "JPEG")
        else:
            os.mkdir(os.path.join(path, 'cropped', 'test', df['folder'][i]))
            im_crop.save(os.path.join(path, 'cropped', 'test', df['folder'][i],
                                     df['name'][i]), "JPEG")
    else:

        if os.path.exists(os.path.join(path, 'cropped', 'train', df['folder'][i])):
            im_crop.save(os.path.join(path, 'cropped', 'train', df['folder'][i],
                                              df['name'][i]), "JPEG")
        else:
            os.mkdir(os.path.join(path, 'cropped', 'train', df['folder'][i]))
            im_crop.save(os.path.join(path, 'cropped', 'train', df['folder'][i],
                                     df['name'][i]), "JPEG")

## Squared Bounding Box

In [ ]:
for i in range(len(df)):

    im_rgb = Image.open(os.path.join(path, 'CUB_200_2011/CUB_200_2011/images', df['folder'][i],
                                     df['name'][i]))
    # square the bounding box, using the side of square as maximum of width or height
    if df['width'][i] < df['height'][i]:
        im_crop = im_rgb.crop(((df['x'][i], df['y'][i],
                          df['x'][i] + df['width'][i],
                          df['y'][i] + df['width'][i])))
    else: 
        im_crop = im_rgb.crop(((df['x'][i], df['y'][i],
                          df['x'][i] + df['height'][i],
                          df['y'][i] + df['height'][i])))
    # store in test set if it is a test image else store in train
    if df['train'][i] == 0: 

        if os.path.exists(os.path.join(path, 'boundariesSqWsH', 'test', df['folder'][i])):
            im_crop.save(os.path.join(path, 'boundariesSqWsH', 'test', df['folder'][i],
                                              df['name'][i]), "JPEG")
        else:
            os.mkdir(os.path.join(path, 'boundariesSqWsH', 'test', df['folder'][i]))
            im_crop.save(os.path.join(path, 'boundariesSqWsH', 'test', df['folder'][i],
                                     df['name'][i]), "JPEG")
    else:

        if os.path.exists(os.path.join(path, 'boundariesSqWsH', 'train', df['folder'][i])):
            im_crop.save(os.path.join(path, 'boundariesSqWsH', 'train', df['folder'][i],
                                              df['name'][i]), "JPEG")
        else:
            os.mkdir(os.path.join(path, 'boundariesSqWsH', 'train', df['folder'][i]))
            im_crop.save(os.path.join(path, 'boundariesSqWsH', 'train', df['folder'][i],
                                     df['name'][i]), "JPEG")

## Segmentation
Grey-out the background of each image.

In [ ]:
df = pd.read_csv("boundaries.csv")

In [ ]:
path = os.getcwd()

In [ ]:
for i in range(len(df)):
    # open the image
    im_rgb = Image.open(os.path.join(path, 'CUB_200_2011/CUB_200_2011/images', df['folder'][i],
                                     df['name'][i])).convert("RGBA")
    # open the corresponding segmentation mask of that image
    im_a = Image.open(os.path.join(path, 'segmentations',
                                   df['folder'][i], 
                                   os.path.splitext(df['name'][i])[0]+".png")).convert('L').resize(im_rgb.size)
    # copy the image
    im_rgba = im_rgb.copy()
    # apply the masking filter to the copy
    im_rgba.putalpha(im_a)
    pixels = im_rgba.load()
    # convert the background filtered image to the grey 
    for y in range(im_rgba.size[1]): 
        for x in range(im_rgba.size[0]):

            if pixels[x,y][3] < 122:    # check alpha
                pixels[x,y] = (122, 122, 122)
    # store in test set if it is a test image else store in train            
    if df['train'][i] == 0: 

        if os.path.exists(os.path.join(path, 'grey', 'test', df['folder'][i])):
            im_rgba.save(os.path.join(path, 'grey', 'test', df['folder'][i],
                                              df['name'][i]), "PNG")
        else:
            os.mkdir(os.path.join(path, 'grey', 'test', df['folder'][i]))
            im_rgba.save(os.path.join(path, 'grey', 'test', df['folder'][i],
                                     df['name'][i]), "PNG")
    else:

        if os.path.exists(os.path.join(path, 'grey', 'train', df['folder'][i])):
            im_rgba.save(os.path.join(path, 'grey', 'train', df['folder'][i],
                                              df['name'][i]), "PNG")
        else:
            os.mkdir(os.path.join(path, 'grey', 'train', df['folder'][i]))
            im_rgba.save(os.path.join(path, 'grey', 'train', df['folder'][i],
                                     df['name'][i]), "PNG")
        
    

## Bounding Box Segmentation
Grey-out the background of each image and make the image square.

In [ ]:
for i in range(len(df)):
    # open the image
    im_rgb = Image.open(os.path.join(path, 'CUB_200_2011/CUB_200_2011/images', df['folder'][i],
                                     df['name'][i])).convert("RGBA")
    # open the corresponding segmentation mask of that image
    im_a = Image.open(os.path.join(path, 'segmentations',
                                   df['folder'][i], 
                                   os.path.splitext(df['name'][i])[0]+".png")).convert('L').resize(im_rgb.size)
    # copy the image
    im_rgba = im_rgb.copy()
    # apply the masking filter to the copy
    im_rgba.putalpha(im_a)
    pixels = im_rgba.load()
    # convert the background filtered image to the grey 
    for y in range(im_rgba.size[1]): 
        for x in range(im_rgba.size[0]):

            if pixels[x,y][3] < 122:    # check alpha
                pixels[x,y] = (122, 122, 122)
    # crop the image as per given co-ordinates and measurements        
    imCrop = im_rgba.crop((df['x'][i], df['y'][i],
                      df['x'][i] + df['width'][i],
                      df['y'][i] + df['height'][i]))
    # store in test set if it is a test image else store in train
    if df['train'][i] == 0: 

        if os.path.exists(os.path.join(path, 'bounding_box_segmentation', 'test', df['folder'][i])):
            imCrop.save(os.path.join(path, 'bounding_box_segmentation', 'test', df['folder'][i],
                                              df['name'][i]), "PNG")
        else:
            os.mkdir(os.path.join(path, 'bounding_box_segmentation', 'test', df['folder'][i]))
            imCrop.save(os.path.join(path, 'bounding_box_segmentation', 'test', df['folder'][i],
                                     df['name'][i]), "PNG")
    else:

        if os.path.exists(os.path.join(path, 'bounding_box_segmentation', 'train', df['folder'][i])):
            imCrop.save(os.path.join(path, 'bounding_box_segmentation', 'train', df['folder'][i],
                                              df['name'][i]), "PNG")
        else:
            os.mkdir(os.path.join(path, 'bounding_box_segmentation', 'train', df['folder'][i]))
            imCrop.save(os.path.join(path, 'bounding_box_segmentation', 'train', df['folder'][i],
                                     df['name'][i]), "PNG")
        
    

## Simple rectification

In [ ]:
# Change training to testing afterwards
folder_location = mirror_dir + 'train/'

for index, row in img_df.iterrows():
    # store in train set if it is a train image else store in test
    if (row['Training'] == 1):      
        for index_df, df_row in df.iterrows():
            # if left eye is present  do nothing
            if (df_row['ID'] == row['ID'] and df_row[1]== 7 and df_row['Visible']== 1):
                print('Left eye present: ' + str(row['Name']))
                break
            # if right eye is present mirror (flip) the image so that all beaks points towards the left
            elif (df_row['ID'] == row['ID'] and df_row[1]==11 and df_row['Visible']== 1):
             
                im = Image.open(folder_location + row['Name'])
                im_mirror = ImageOps.mirror(im)
                im_mirror.save(folder_location + row['Name'])

                break